In [1]:
import argparse
import glob
import os
import xarray as xr
from pathlib import Path


In [3]:
def create_ensemble_zarr(base_dir, output_zarr_path):
    """
    Reads a set of NetCDF files from ensemble directories, adds an ensemble dimension,
    and stores the combined dataset as a single Zarr file.

    Parameters:
    - base_dir: str
        The base directory containing date-stamped folders with ensemble subdirectories.
        e.g., "forecast/output/ensembles/<date>"
    - output_zarr_path: str
        The file path for the output Zarr file.
    """
    # For each date directory, list ensemble subdirectories
    ensemble_dirs = sorted(glob.glob(os.path.join(base_dir, "ensemble_*")))
    
    # List to hold datasets for each ensemble member for this date
    ensemble_datasets = []
    
    for ens_dir in ensemble_dirs:
        # Use xarray to open all NetCDF files in the ensemble directory.
        # This assumes that all files in an ensemble folder should be combined.
        # If not, refine the glob pattern to target only relevant files.
        file_pattern = os.path.join(ens_dir, "*.nc")
        ds = xr.open_mfdataset(file_pattern, combine='by_coords')
        
        # Extract ensemble index from folder name, e.g., ensemble_0 -> 0
        ens_index = int(os.path.basename(ens_dir).split('_')[-1])
        # Expand dataset along a new "ensemble" dimension with coordinate value
        ds = ds.expand_dims(ensemble=[ens_index])
        
        ensemble_datasets.append(ds)
    
    # Concatenate all ensemble datasets along the new ensemble dimension
    if ensemble_datasets:
        combined_ds = xr.concat(ensemble_datasets, dim='ensemble')
        sorted_ds = combined_ds.sortby('ensemble')

    # Write the final dataset to a single Zarr file
    sorted_ds.to_zarr(output_zarr_path, mode='w')
    print(f"Ensemble Zarr file created at: {output_zarr_path}")


In [53]:
import os
import glob
import xarray as xr
import zarr

def create_ensemble_zarr(base_dir, output_zarr_path):
    """
    Reads a set of NetCDF files from ensemble directories, adds an ensemble dimension,
    and stores the combined dataset as a single Zarr file.

    Parameters:
    - base_dir: str
        The base directory containing date-stamped folders with ensemble subdirectories.
        e.g., "forecast/output/ensembles/<date>"
    - output_zarr_path: str
        The file path for the output Zarr file.
    """
    compressor = zarr.Blosc(cname="zstd", clevel=3, shuffle=2)


    # For each date directory, list ensemble subdirectories sorted numerically
    ensemble_dirs = sorted(
        glob.glob(os.path.join(base_dir, "ensemble_*")),
        key=lambda x: int(os.path.basename(x).split('_')[-1])
    )

    # Create an empty dataset to initialize the Zarr store
    zarr_initialized = False

    for ens_dir in ensemble_dirs:
        # Use xarray to open all NetCDF files in the ensemble directory.
        file_pattern = os.path.join(ens_dir, "*.nc")
        ds = xr.open_mfdataset(file_pattern, combine='by_coords', parallel=True, chunks={})

        # Extract ensemble index from folder name, e.g., ensemble_0 -> 0
        ens_index = int(os.path.basename(ens_dir).split('_')[-1])
        ds = ds.expand_dims(ensemble=[ens_index])
        # Example encoding for variables
        # Define encoding only for the initial write
        encoding = {var: {"compressor": compressor} for var in ds.data_vars}
        if not zarr_initialized:
            # Initialize the Zarr store
            ds.to_zarr(output_zarr_path, mode='w', consolidated=True, encoding=encoding, compute=True)
            zarr_initialized = True
        else:
            # Append to the existing Zarr store
            ds.to_zarr(output_zarr_path, mode='a', consolidated=True, append_dim='ensemble', compute=True)

        print(f"Processed ensemble: {ens_index}")

    print(f"Ensemble Zarr file created at: {output_zarr_path}")


In [54]:
project_dir = Path("/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/forecast/output")
input_dir = project_dir / "ensembles/2025-01-21"
zarr_file = input_dir / f"{input_dir.name}.zarr"

create_ensemble_zarr(base_dir=input_dir, output_zarr_path=zarr_file)

Processed ensemble: 0
Processed ensemble: 1
Processed ensemble: 2
Processed ensemble: 3
Processed ensemble: 4
Processed ensemble: 5
Processed ensemble: 6
Processed ensemble: 7
Processed ensemble: 8
Processed ensemble: 9
Processed ensemble: 10
Processed ensemble: 11
Processed ensemble: 12
Processed ensemble: 13
Processed ensemble: 14
Processed ensemble: 15
Processed ensemble: 16
Processed ensemble: 17
Processed ensemble: 18
Processed ensemble: 19
Processed ensemble: 20
Processed ensemble: 21
Processed ensemble: 22
Processed ensemble: 23
Processed ensemble: 24
Processed ensemble: 25
Processed ensemble: 26
Processed ensemble: 27
Processed ensemble: 28
Processed ensemble: 29
Processed ensemble: 30
Processed ensemble: 31
Processed ensemble: 32
Processed ensemble: 33
Processed ensemble: 34
Processed ensemble: 35
Processed ensemble: 36
Processed ensemble: 37
Processed ensemble: 38
Processed ensemble: 39
Processed ensemble: 40
Processed ensemble: 41
Processed ensemble: 42
Processed ensemble: 4

In [32]:
ds_t = xr.open_dataset(
    '/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/forecast/output/ensembles/2025-01-21/ensemble_0/20250217_tmaxf.nc'
)
ds_t['tmaxf'].values

array([[14.0257, 37.9   , 37.9   , ..., 15.8396, 12.1   , 12.9   ],
       [19.0257, 33.9   , 33.8   , ..., 20.8396, 17.5   , 18.    ],
       [20.6257, 46.9   , 46.9   , ..., 22.5396, 19.2   , 19.6   ],
       ...,
       [21.896 , 51.5   , 51.7   , ..., 21.5   , 19.    , 19.9   ],
       [22.796 , 56.4   , 56.2   , ..., 22.4   , 19.5   , 20.8   ],
       [21.996 , 55.    , 55.    , ..., 21.5   , 18.8   , 20.    ]],
      dtype=float32)

In [56]:
import numpy as np
ds_new = xr.open_zarr(zarr_file)
ds_new
np.nanmax(ds_new['potet'].sel(ensemble=10, time="2025-01-21").values)

4.8768